# Data Ingestion
## Extract files from source raw

Unzip documents & Combine 187k csv to one 

original data source:
- https://drive.google.com/file/d/1AmGcOs5U31wIIqvthn9ARqJMrMTFTcaw/view
- https://drive.google.com/file/d/117aYJAWG3GU48suS66NPaB82HwFj6xWS/view

processed data folder: https://drive.google.com/drive/folders/1SxL76uErvzq7PZGXCZYqMJE0sW7dWUsM?usp=drive_link
- temp.csv: processed through this notebook
- quesions.csv: downloaded from second original source
- lectures.csv: downloaded from second original source
Since the unzip process takes too long, We decided to manually stop the unzip and collect the filenames of unzipped ones for reproduction purpose. 
- keys.txt: names of csv used as a subset in this project.

All data files needed for later stages are included in the above processed data folder.

Env: google colab

In [ ]:
#!pip install patool
import patoolib
patoolib.extract_archive('/content/EdNet-KT1.zip')

In [ ]:
#The section below processes 187k csv files with each of the file name as a unique student id.
import os, re
import pandas as pd
unp = os.listdir('/content/')
folder = [s for s in unp if s.startswith("Unpack")][0]

dir = '/content/' + folder + '/KT1'
dfs = []
rem = []
for filename in os.listdir(dir):
    if filename.endswith('.csv'):
        # Extract user_id from filename, assuming it's formatted like 'uXXXXX.csv'
        user_id = re.search(r'u(\d+)', filename).group(1)

        # Read the CSV file
        file_path = os.path.join(dir, filename)
        try:
            df = pd.read_csv(file_path)

            # Check if the dataframe is empty
            if df.empty:
                print(f"Skipping empty file: {filename}")
                rem.append(filename)
                continue

            # Add the user_id column
            df['user_id'] = user_id

            # Append the dataframe to the list
            dfs.append(df)
        except pd.errors.EmptyDataError:
            print(f"Skipping empty file: {filename}")
            rem.append(filename)
            continue


# Concatenate all dataframes into one
combined_df = pd.concat(dfs, ignore_index=True)

Skipping empty file: u822177.csv


In [ ]:
combined_df.to_csv('/content/temp.csv')
file_size = os.path.getsize('/content/temp.csv')
print(file_size)

1075645401


In [ ]:
#document the filenames that is extracted in keys.txt
selected_files = os.listdir(dir)
#
for remo in rem:
  selected_files.remove(remo)
# Save the list of selected file names to a text file for future reference
with open('keys.txt', 'w') as f:
    for filename in selected_files:
        f.write(f"{filename}\n")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Save dataframe to google drive
file_path = '/content/drive/My Drive/5400Project/data_vsubmit/temp.csv'
# Save the DataFrame to a CSV file
combined_df.to_csv(file_path)